## DIA 050: Implementacion de WebSockets para Notificaciones en Tiempo Real

Hoy implementaremos WebSockets en nuestra API para enviar notificaciones en tiempo real a los usuarios. Esto permitirá que la aplicación informe a los clientes sobre eventos importantes como:
✅ Actualización de predicciones en vivo
✅ Alertas de mantenimiento del sistema
✅ Mensajes personalizados desde administradores

Utilizaremos Flask-SocketIO para manejar conexiones WebSocket en la API. Los clientes podrán suscribirse y recibir notificaciones sin necesidad de hacer peticiones constantes al servidor.

🖥️ Código Completo (api.py)
python
Copiar
Editar
import os
import json
import logging
from datetime import datetime

from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_socketio import SocketIO, emit, join_room, leave_room

# Configuración básica de la API
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'jwtsecretkey')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de Extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
jwt = JWTManager(app)
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos de la Base de Datos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)

class Notification(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    message = db.Column(db.Text, nullable=False)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

# ---------------------------
# Función para Enviar Notificaciones
# ---------------------------
def send_notification(username, message):
    """Envía una notificación a un usuario específico en tiempo real."""
    notification = Notification(username=username, message=message)
    db.session.add(notification)
    db.session.commit()
    logger.info(f"Notificación enviada a {username}: {message}")
    
    # Emitir mensaje vía WebSocket
    socketio.emit(f'notification_{username}', {"message": message}, broadcast=True)

# ---------------------------
# Endpoints de la API
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    Iniciar sesión y obtener un token JWT.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')  # Aquí debería validarse con base de datos
    
    if not username or not password:
        return jsonify({"msg": "Credenciales incorrectas"}), 400
    
    token = create_access_token(identity=username)
    return jsonify(access_token=token), 200

@app.route('/notify', methods=['POST'])
@jwt_required()
def notify():
    """
    Enviar notificación a un usuario autenticado.
    """
    data = request.get_json()
    current_user = get_jwt_identity()
    message = data.get("message", "Notificación sin contenido")

    send_notification(current_user, message)
    return jsonify({"msg": "Notificación enviada"}), 200

@app.route('/admin/broadcast', methods=['POST'])
@jwt_required()
def admin_broadcast():
    """
    Enviar una notificación a todos los usuarios (solo administradores).
    """
    data = request.get_json()
    message = data.get("message", "Mensaje de administrador")

    users = User.query.all()
    for user in users:
        send_notification(user.username, message)
    
    return jsonify({"msg": "Notificación enviada a todos los usuarios"}), 200

@app.route('/notifications', methods=['GET'])
@jwt_required()
def get_notifications():
    """
    Obtener las notificaciones de un usuario autenticado.
    """
    current_user = get_jwt_identity()
    notifications = Notification.query.filter_by(username=current_user).order_by(Notification.timestamp.desc()).all()
    return jsonify([{"message": n.message, "timestamp": n.timestamp.isoformat()} for n in notifications]), 200

# ---------------------------
# Eventos de WebSocket
# ---------------------------
@socketio.on('connect')
def handle_connect():
    """Evento de conexión WebSocket."""
    logger.info("Un cliente se ha conectado.")

@socketio.on('disconnect')
def handle_disconnect():
    """Evento de desconexión WebSocket."""
    logger.info("Un cliente se ha desconectado.")

@socketio.on('join')
def handle_join(data):
    """Un usuario se une a su canal de notificaciones."""
    username = data.get("username")
    if username:
        join_room(username)
        logger.info(f"{username} se ha unido a su canal de notificaciones.")
        emit('join_response', {"msg": f"Conectado al canal de notificaciones de {username}."}, room=username)

@socketio.on('leave')
def handle_leave(data):
    """Un usuario sale de su canal de notificaciones."""
    username = data.get("username")
    if username:
        leave_room(username)
        logger.info(f"{username} salió de su canal de notificaciones.")
        emit('leave_response', {"msg": f"Desconectado del canal de notificaciones de {username}."}, room=username)

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
🔍 Explicación de las Principales Implementaciones
🔹 📡 WebSockets con Flask-SocketIO

Permite comunicación en tiempo real entre el servidor y los clientes.
Se implementan eventos para conexión, desconexión, unión y salida de canales.
🔹 🔔 Notificaciones Personalizadas

Los usuarios pueden recibir notificaciones en vivo a través de WebSockets.
Se almacena un historial de notificaciones en la base de datos.
🔹 🔑 Protección con JWT

Solo usuarios autenticados pueden acceder a las notificaciones.
Se impide que usuarios no autorizados envíen notificaciones a otros.
🔹 📩 Endpoints Implementados
✅ /notify → Enviar una notificación a un usuario autenticado.
✅ /admin/broadcast → Enviar un mensaje global a todos los usuarios (solo administradores).
✅ /notifications → Obtener el historial de notificaciones de un usuario.

🔹 📢 Eventos de WebSocket
✅ connect → Un cliente se conecta al sistema.
✅ disconnect → Un cliente se desconecta del servidor.
✅ join → Un usuario se une a su canal de notificaciones.
✅ leave → Un usuario deja su canal de notificaciones.